In [127]:
import pandas as pd
#from urllib.request import urlretrieve
#import plotly.express as px
import matplotlib
import matplotlib.pyplot as plt
import torchvision
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
#from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
from torch.utils.data import Dataset, DataLoader, random_split
import torch.optim as optim
import time
import torch
import glob
from sklearn.preprocessing import StandardScaler

In [128]:
torch.cuda.is_available()
def get_device():
    # Step 2: Check if a GPU is available
    if torch.cuda.is_available():
        # Step 3: Set the device to GPU
        device = torch.device("cuda")
        print("GPU is available. Using GPU.")
    else:
        # Step 3: Default to CPU
        device = torch.device("cpu")
        print("GPU is not available. Using CPU.")
    
    # Step 4: Return the device
    return device

# Example usage
device = get_device()

GPU is available. Using GPU.


In [129]:
class ThreeLayerNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ThreeLayerNN, self).__init__()
        # Define the layers
        self.fc1 = nn.Linear(input_size, hidden_size)  # First fully connected layer
        self.relu1 = nn.ReLU()  # Activation function
        self.fc2 = nn.Linear(hidden_size, hidden_size)  # Second fully connected layer
        self.relu2 = nn.ReLU()  # Activation function
        self.fc3 = nn.Linear(hidden_size, output_size)  # Third fully connected layer

    def forward(self, x):
        # Forward pass
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

In [130]:

#num_samples = 1000
#num_instances = 10000


#x = torch.rand(num_instances, num_samples)
#y = torch.rand(num_instances, num_samples)
#z = torch.rand(num_instances, num_samples)
#target = torch.randint(-1000, 1001, (num_instances, num_samples))
#data = torch.stack((x, y, z, target), dim=2).to(device)
# Define the pattern to match files
start_time = time.time()
file_pattern = 'SWPR_WDFC.wRes.nom_SP.058_SP.060_2D_w0(*).xlsx'

# Find all files matching the pattern
file_paths = glob.glob(file_pattern)

# Initialize a list to store the tensors
data_list = []

# Loop through the list of matching files
for file_path in file_paths:
    columns_to_read = ['E_X', 'E_Y', 'E_Z', 'PSCL']
    df = pd.read_excel(file_path, usecols=columns_to_read)
    data_array = df.to_numpy()
    data_tensor = torch.tensor(data_array, dtype=torch.float32)
    data_list.append(data_tensor)

# Determine the maximum size of the second dimension
max_size = max(tensor.shape[0] for tensor in data_list)

# Pad each tensor to the maximum size
padded_data_list = []
for tensor in data_list:
    padding = (0, 0, 0, max_size - tensor.shape[0])
    padded_tensor = torch.nn.functional.pad(tensor, padding, mode='constant', value=0)
    padded_data_list.append(padded_tensor)

# Stack the padded tensors into a single tensor
data_unscaled = torch.stack(padded_data_list)

# Print the shape of the final tensor

print(data_unscaled.shape)

#data scaling+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
data_2d = data_unscaled.view(-1, 4).numpy()
# Initialize the scaler
scaler = StandardScaler()

# Fit the scaler on the 2D data
scaler.fit(data_2d)
# Transform the data
scaled_data_2d = scaler.transform(data_2d)
# Reshape the scaled data back to 3D
data = torch.tensor(scaled_data_2d).view(len(file_paths), max_size, 4)



end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds for reading data")
print(data.shape)





torch.Size([10, 5670, 4])
Elapsed time: 39.08772253990173 seconds for reading data
torch.Size([10, 5670, 4])


In [131]:
print("origincal data:")
print(data_unscaled)
print("Scaled data:")
print(data)

origincal data:
tensor([[[ 3677.6001,   177.7600,     0.0000, 33179.8555],
         [ 3677.5283,   178.0630,     0.0000, 33179.8555],
         [ 3677.4565,   178.3660,     0.0000, 33179.8555],
         ...,
         [ 3623.3999,   120.3637,     0.0000, 31282.0039],
         [ 3623.3999,   120.2662,     0.0000, 31303.5430],
         [ 3623.3999,   120.1687,     0.0000, 31324.8691]],

        [[ 3677.6001,   177.7600,     0.0000, 33179.8555],
         [ 3677.5283,   178.0630,     0.0000, 33179.8555],
         [ 3677.4565,   178.3660,     0.0000, 33179.8555],
         ...,
         [ 3623.3999,   120.3637,     0.0000, 31282.0039],
         [ 3623.3999,   120.2662,     0.0000, 31303.5430],
         [ 3623.3999,   120.1687,     0.0000, 31324.8691]],

        [[ 3677.6001,   177.7600,     0.0000, 33179.8555],
         [ 3677.5283,   178.0630,     0.0000, 33179.8555],
         [ 3677.4565,   178.3660,     0.0000, 33179.8555],
         ...,
         [ 3623.3999,   120.2662,     0.0000, 31303.5

In [132]:
val_size = len(data_list)//10
train_size = len(data_list) - val_size
print(val_size)
print(train_size)
start_time = time.time()
train_ds, val_ds = random_split(data, [train_size, val_size])
batch_size=2
train_loader = DataLoader(train_ds, batch_size, shuffle=True, num_workers=0)
val_loader = DataLoader(val_ds, batch_size*2, num_workers=0)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds for manipulating data.")

1
9
Elapsed time: 0.0009980201721191406 seconds for manipulating data.


In [133]:
for batch in train_loader:
    print(batch.shape)
    inputs =    batch[:,...,:-1]
    targets =   batch[:,...,-1]
    print(inputs)
    print(targets)   # Forward pass
    break

torch.Size([2, 5670, 4])
tensor([[[ 1.3760,  0.7974,  0.0000],
         [ 1.3746,  0.8036,  0.0000],
         [ 1.3733,  0.8097,  0.0000],
         ...,
         [ 0.3394, -0.3686,  0.0000],
         [ 0.3394, -0.3706,  0.0000],
         [ 0.3394, -0.3726,  0.0000]],

        [[ 1.3760,  0.7974,  0.0000],
         [ 1.3746,  0.8036,  0.0000],
         [ 1.3733,  0.8097,  0.0000],
         ...,
         [ 0.3394, -0.3686,  0.0000],
         [ 0.3394, -0.3706,  0.0000],
         [ 0.3394, -0.3726,  0.0000]]])
tensor([[0.4929, 0.4929, 0.4929,  ..., 0.0476, 0.0527, 0.0577],
        [0.4929, 0.4929, 0.4929,  ..., 0.0476, 0.0527, 0.0577]])


In [134]:
#model = nn.Linear(100 * 3, 100)

In [135]:
#model.parameters()

In [136]:
#data[:,...,:-1].shape

In [137]:
model = ThreeLayerNN(3, 100, 1).to(device)

In [138]:
#model.parameters()

In [139]:
optimizer = optim.Adam(model.parameters(), lr=1e-5)

In [142]:


# Assuming model, train_loader, and criterion are already defined
#model = ...  # Your neural network model
#train_loader = ...  # DataLoader for your training data
criterion = nn.MSELoss()  # Loss function for regression

num_epochs = 10000  # Number of epochs

# Training loop
start_time = time.time()
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    for batch in train_loader:
        optimizer.zero_grad()  # Zero the gradients
        inputs = batch[:,...,:-1].to(device)
        outputs = model(inputs).to(device)
        targets =   batch[:,...,-1].to(device)   # Forward pass
        outputs = outputs.squeeze(-1)
        loss = criterion(outputs, targets)  # Compute the loss

        loss.backward()  # Backward pass
        optimizer.step()  # Update the weights

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}")
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds for {len(data_list)} simulations with {len(data[1])} points each on {device}.")


Epoch [1/10000], Loss: 0.9278654813766479
Epoch [2/10000], Loss: 0.9272732615470887
Epoch [3/10000], Loss: 0.926686429977417
Epoch [4/10000], Loss: 0.9261047601699829
Epoch [5/10000], Loss: 0.9255279541015625
Epoch [6/10000], Loss: 0.9249559640884399
Epoch [7/10000], Loss: 0.9243895769119262
Epoch [8/10000], Loss: 0.9238286018371582
Epoch [9/10000], Loss: 0.923272716999054
Epoch [10/10000], Loss: 0.9227219939231872
Epoch [11/10000], Loss: 0.9221766233444214
Epoch [12/10000], Loss: 0.9216365218162537
Epoch [13/10000], Loss: 0.9211012601852417
Epoch [14/10000], Loss: 0.9205703377723694
Epoch [15/10000], Loss: 0.9200439095497132
Epoch [16/10000], Loss: 0.9195222139358521
Epoch [17/10000], Loss: 0.9190053462982177
Epoch [18/10000], Loss: 0.9184932470321655
Epoch [19/10000], Loss: 0.9179858565330505
Epoch [20/10000], Loss: 0.9174828648567199
Epoch [21/10000], Loss: 0.9169841170310974
Epoch [22/10000], Loss: 0.9164895296096802
Epoch [23/10000], Loss: 0.915998911857605
Epoch [24/10000], Loss:

In [147]:
model.eval()  # Set the model to evaluation mode
val_loss = 0.0
start_time = time.time()
with torch.no_grad():
    for batch in val_loader:
        inputs = batch[:, ..., :-1].to(device)
        outputs = model(inputs).to(device)
        targets = batch[:, ..., -1].to(device)  # Forward pass
        outputs = outputs.squeeze(-1)
        loss = criterion(outputs, targets).to(device)  # Compute the loss
        val_loss += loss.item()
end_time = time.time()

elapsed_time = end_time - start_time
print(f"Epoch [{epoch+1}/{num_epochs}], Validation Loss: {val_loss/len(val_loader)}")
print(f"validation took {elapsed_time} secondson {device}.")

Epoch [10000/10000], Validation Loss: 10.777233123779297
validation took 0.0 secondson cuda.
